## Hugging Face Code

In [ ]:
hugging_face_1bLLamaInstruct = "token"

In [2]:
from huggingface_hub import login

login(hugging_face_1bLLamaInstruct)

/home/galnoy/git-projects/MSC-Text-Retrieval-and-Search-Engines/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# java installation

In [3]:
!apt-get update
!apt-get install -y openjdk-21-jdk
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/java-21-openjdk-amd64/bin/java 1
!update-alternatives --install /usr/bin/javac javac /usr/lib/jvm/java-21-openjdk-amd64/bin/javac 1
!update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-21-openjdk-amd64/bin/javac

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
update-alternatives: error: unable to create file '/var/lib/dpkg/alternatives/java.dpkg-tmp': Permission denied
update-alternatives: error: unable to create file '/var/lib/dpkg/alternatives/javac.dpkg-tmp': Permission denied


# install torch

In [4]:

!pip install torch torchvision torchaudio

# install Faiss


In [ ]:
!pip install faiss-cpu --no-cache

# install Pyserini


In [ ]:
!pip install pyserini==0.36.0

# Load Index - KILT Benchmark

In [5]:
from pyserini.search import SimpleSearcher
searcher = SimpleSearcher.from_prebuilt_index('wikipedia-kilt-doc')

2025-12-12 11:04:39.495567925 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"
Dec 12, 2025 11:04:40 AM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [6]:
from pyserini.index.lucene import IndexReader
index_reader = IndexReader.from_prebuilt_index('wikipedia-kilt-doc')

In [7]:
print(index_reader.stats())

{'total_terms': 1915061164, 'documents': 5903530, 'non_empty_documents': 5903529, 'unique_terms': 8722502}


# Load Queries
#### (Train (3778 questions) and Test (2032 questions) files are given as part of this exercise in a csv format)

In [8]:
import pandas as pd
import json
df_train = pd.read_csv("data/train.csv", converters={"answers": json.loads})
df_test = pd.read_csv("data/test.csv", converters={"answers": json.loads})

In [9]:
df_train.head()

,question,answers,id
0,what is the name of justin bieber brother?,"[Jazmyn Bieber, Jaxon Bieber]",1
1,what character did natalie portman play in sta...,[Padmé Amidala],2
2,what state does selena gomez?,[New York City],3
3,what country is the grand bahama island in?,[Bahamas],4
4,what kind of money to take to bahamas?,[Bahamian dollar],5


In [10]:
df_test.head()

,question,id
0,what does jamaican people speak?,1
1,what did james k polk do before he was president?,2
2,what is the oregon ducks 2012 football schedule?,3
3,who plays ken barlow in coronation street?,4
4,what happened after mr. sugihara died?,5


# Load LLM (Llama-3.2-1B-Instruct)

In [11]:
import transformers
import torch


model_id = "meta-llama/Llama-3.2-1B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


# Example for retrieval to be used as few Shots (Context) for LLM



#### Retrieval

In [12]:
searcher.set_qld(mu=1000)

In [13]:
import json
def get_context (query, k=5):
  context = []
  hits = searcher.search(query, k)

  # Get text from hits
  for hit in hits:
      doc = searcher.doc(hit.docid)
      raw_json = doc.raw()
      data = json.loads(raw_json)
      contents = data['contents']
      content = contents.replace('\n', ' ')[:300]  # Truncate for clarity
      context.append(content)
  return context

In [14]:
def create_message (query, contexts):
  # We adapted Tamber et al.’s prompt Tamber et al. [Benchmarking LLM Faithfulness in RAG with Evolving Leaderboards.], used for their LLM-as-a-judge approach – Faith- Judge (https://github.com/vectara/FaithJudge)
  context = '\n'.join(contexts)
  messages = [
    {"role": "system", "content": "You must respond based strictly on the information in provided passages. Do not incorporate any external knowledge or infer any details beyond what is given in the passages. If the answer is not in the context, return 'I dont know'. Do not include explanations, only the final answer!"},
    {"role": "user", "content": f'Provide a concise answer to the following question based on the information in the provided documents. Documents: {context}. *Question: {question}*. Answer: '},
  ]
  return messages

In [15]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [16]:
def llm_answer (query):
  contexts = get_context(query)
  messages = create_message(query, contexts)
  print(messages)
  outputs = pipeline(
      messages,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  answer = (outputs[0]["generated_text"][-1]).get('content')
  print(answer)
  return answer

In [17]:
predictions_LLM = {}
# loop over the first 100 test queries
for index, row in df_test.head(100).iterrows():
  question = row['question']
  print(f"Question: {question}")
  predictions_LLM[row['id']] = llm_answer(question)
  print("-" * 20)

Question: what does jamaican people speak?


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[{'role': 'system', 'content': "You must respond based strictly on the information in provided passages. Do not incorporate any external knowledge or infer any details beyond what is given in the passages. If the answer is not in the context, return 'I dont know'. Do not include explanations, only the final answer!"}, {'role': 'user', 'content': 'Provide a concise answer to the following question based on the information in the provided documents. Documents: Jamaica Labrish Jamaica Labrish is a poetry compilation written by Louise Bennett-Coverley. The 1966 version published by Sangsters is 244 pages long with an introduction by Rex Nettleford and includes a four-page glossary, as the poems are written mainly in Jamaican Patois. There are 128 poems in t\nLGBT rights in Jamaica Lesbian, gay, bisexual, and transgender (LGBT) persons in Jamaica, especially men, face legal and social issues not experienced by non-LGBT people. Sodomy and/or buggery are punishable by imprisonment. On the oth

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[{'role': 'system', 'content': "You must respond based strictly on the information in provided passages. Do not incorporate any external knowledge or infer any details beyond what is given in the passages. If the answer is not in the context, return 'I dont know'. Do not include explanations, only the final answer!"}, {'role': 'user', 'content': 'Provide a concise answer to the following question based on the information in the provided documents. Documents: James K. Polk (song) "James K. Polk" is a song by alternative rock band They Might Be Giants, about the United States president of the same name. Originally released in 1990 as a B-side to the single "Istanbul (Not Constantinople)", its first appearance on a studio album was 1996\'s "Factory Showroom\nPresident James K. Polk Home &amp; Museum The President James K. Polk Home & Museum is the Presidential Museum for 11th President James K. Polk, located at 301 West 7th Street in Columbia, Tennessee. Built in 1816, it is the only surv

...

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[{'role': 'system', 'content': "You must respond based strictly on the information in provided passages. Do not incorporate any external knowledge or infer any details beyond what is given in the passages. If the answer is not in the context, return 'I dont know'. Do not include explanations, only the final answer!"}, {'role': 'user', 'content': "Provide a concise answer to the following question based on the information in the provided documents. Documents: International reactions to the Fukushima Daiichi nuclear disaster The international reaction to the 2011 Fukushima Daiichi nuclear disaster has been diverse and widespread. Many inter-governmental agencies responded to the Japanese Fukushima Daiichi nuclear disaster, often on an ad hoc basis. Respon\nAnti-nuclear power movement in Japan Long one of the world's most committed promoters of civilian nuclear power, Japan's nuclear industry was not hit as hard by the effects of the 1979 Three Mile Island accident (USA) or the 1986 Chern

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Ōkuma and Futaba.
--------------------
Question: who is the minority leader of the house of representatives now?
[{'role': 'system', 'content': "You must respond based strictly on the information in provided passages. Do not incorporate any external knowledge or infer any details beyond what is given in the passages. If the answer is not in the context, return 'I dont know'. Do not include explanations, only the final answer!"}, {'role': 'user', 'content': 'Provide a concise answer to the following question based on the information in the provided documents. Documents: Party leaders of the United States House of Representatives Party leaders and whips of the United States House of Representatives, also known as floor leaders, are elected by their respective parties in a closed-door caucus by secret ballot. With the Democrats holding a majority of seats and the Rep\nList of members of the American Legislative Exchange Council The American Legislative Exchange Council, otherwise known by

KeyboardInterrupt: 

### Evaluation

In the evaluation check if the answer is contained in the generated response. E.g., [Natalie Portman played Padmé Amidala] and the answer is just [Padmé Amidala].

In [ ]:
import json
import re
import string
from collections import Counter

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        return ''.join(ch for ch in text if ch not in set(string.punctuation))

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    """Compute token-level F1 score between prediction and a ground truth."""
    pred_tokens = normalize_answer(prediction).split()
    gt_tokens = normalize_answer(ground_truth).split()
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())

    if len(pred_tokens) == 0 or len(gt_tokens) == 0:
        return int(pred_tokens == gt_tokens)
    if num_same == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    return max(metric_fn(prediction, gt) for gt in ground_truths)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str = 'id') -> float:
    """
    Computes average F1 score over all questions. Used for leaderboard ranking.
    """

    # Align dataframes and remove ID column
    gold = solution.set_index(row_id_column_name)
    pred = submission.set_index(row_id_column_name)

    f1_sum = 0.0
    count = 0

    for qid in gold.index:
        if qid not in pred.index:
            print(f"Missing prediction for question ID: {qid}")
            count+=1
            continue

        try:
            ground_truths = json.loads(gold.loc[qid, "answers"])
            if not isinstance(ground_truths, list):
                raise ValueError
        except Exception:
            raise Exception(f"Invalid format for answers at id {qid}: must be a JSON list of strings.")

        prediction = pred.loc[qid, "prediction"]
        f1 = metric_max_over_ground_truths(f1_score, prediction, ground_truths)

        f1_sum += f1
        count += 1

    if count == 0:
        raise Exception("No matching question IDs between submission and solution.")

    return 100.0 * f1_sum / count

# ------------------------
# Example usage:
# ------------------------

# Load gold dataset
gold = {
    'id': [1,2,3],
    'answers': ['["Paris"]', '["William Shakespeare", "Shakespeare"]', '["Jupiter"]'],
}

gold_df = pd.DataFrame(gold)

# Load predictions
predictions = {
    'id': [1,2,3],
    'prediction': ["Paris", "Shakespeare", "Earth"]
}

predictions_df = pd.DataFrame(predictions)
predictions_df["prediction"] = predictions_df["prediction"].apply(lambda x: json.dumps([x], ensure_ascii=False))

# Evaluate
results = score(gold_df, predictions_df)
print(f"F1 Score: {results:.2f}")

In [ ]:
df_prediction = pd.DataFrame(predictions_LLM.items(), columns=['id', 'prediction'])
df_prediction["prediction"] = df_prediction["prediction"].apply(lambda x: json.dumps([x], ensure_ascii=False))
df_gold = pd.read_csv("data/train.csv")
df_gold["answers"] = df_gold["answers"].apply(lambda x: json.dumps([x], ensure_ascii=False))


print(score(df_gold, df_prediction))

In [ ]:
df_prediction.to_csv("data/predictions_LLM_baseline_qld.csv", index=False)